In [1]:
import numpy as np
import argparse
import random
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler
import torch.utils.data
from scipy.io import loadmat
from model.GMVAE import *
import matplotlib.pyplot as plt
import os
import glob

In [6]:
# Directory containing the files
folder_path = 'checkpoint'

# Construct the pattern to match files ending with '.pth'
file_pattern = os.path.join(folder_path, '*.pth')

# Use glob to find files matching the pattern
matching_files = glob.glob(file_pattern)

# Print the list of matching files
print("List of files ending with '.pth':")
for file in matching_files:
    print(file)
    checkpoint = torch.load(file, map_location=torch.device('cpu'))
    args = params = checkpoint['config']
    print(params.desc)


List of files ending with '.pth':
checkpoint/model_3.pth
> /tmp/ipykernel_2473115/3387544945.py(12)<module>()
     10 # Print the list of matching files
     11 print("List of files ending with '.pth':")
---> 12 for file in matching_files:
     13     print(file)
     14     checkpoint = torch.load(file, map_location=torch.device('cpu'))

*** TypeError: 'Namespace' object is not subscriptable
Namespace(file=None, dataset='cifar10', seed=0, cuda=1, gpuID=0, epochs=10, batch_size=64, batch_size_val=200, learning_rate=0.001, decay_epoch=-1, lr_decay=0.5, num_classes=10, gaussian_size=64, input_size=3072, train_proportion=1.0, init_temp=1.0, decay_temp=1, hard_gumbel=0, min_temp=0.5, decay_temp_rate=0.013862944, w_gauss=1, w_categ=1, w_rec=1, rec_type='mse', verbose=0, random_search_it=20, save_file='model_3', desc='cifar10')


In [ ]:
model_name = 'model_3'

checkpoint = torch.load(f"checkpoint/{model_name}.pth", map_location=torch.device('cpu'))
args = params = checkpoint['config']

In [ ]:
## Random Seed
SEED = args.seed
np.random.seed(SEED)
random.seed(SEED)
torch.manual_seed(SEED)
if args.cuda:
  torch.cuda.manual_seed(SEED)

In [ ]:
dataset_dict = {
    'mnist': datasets.MNIST,
    'cifar10': datasets.CIFAR10,
    'cifar100': datasets.CIFAR100
}

print(f"Loading {args.dataset} dataset...")
# Download or load downloaded MNIST dataset
train_dataset = dataset_dict[args.dataset](f'./data/{args.dataset}', train=True, download=True, transform=transforms.ToTensor())
test_dataset = dataset_dict[args.dataset](f'./data/{args.dataset}', train=False, transform=transforms.ToTensor())


In [ ]:
def partition_dataset(n, proportion=0.8):
  train_num = int(n * proportion)
  indices = np.random.permutation(n)
  train_indices, val_indices = indices[:train_num], indices[train_num:]
  return train_indices, val_indices

if args.train_proportion == 1.0:
  # we use all train dataset without partitioning
  train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True)
  test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=args.batch_size_val, shuffle=False)
  val_loader = test_loader
else:
  # partition dataset according to train_proportion
  train_indices, val_indices = partition_dataset(len(train_dataset), args.train_proportion)
  # create data loaders for train, validation and test datasets
  train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=args.batch_size, sampler=SubsetRandomSampler(train_indices))
  val_loader = torch.utils.data.DataLoader(train_dataset, batch_size=args.batch_size_val, sampler=SubsetRandomSampler(val_indices))
  test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=args.batch_size_val, shuffle=False)


## Train Model

In [ ]:
print(f"params: {params}")
loss_history = checkpoint['loss_history']
print(loss_history.keys())
plt.plot(np.array(loss_history['train_history_nmi']), label='Train')
plt.plot(np.array(loss_history['val_history_nmi']), label='Validation')

# plt.plot(loss_history, label='KLD')
plt.ylim([0, None])
plt.title(f"Training: NMI score")
plt.xlabel("Epochs")
# plt.ylabel("KLD loss")
plt.legend()
plt.show()

In [ ]:
# Model Initialization
params.cuda=False
gmvae = GMVAE(params)
gmvae.network.load_state_dict(checkpoint['model_state'])

## Test Data

In [ ]:
accuracy, nmi = gmvae.test(test_loader)

print("Testing phase...")
print("Accuracy: %.5lf,  NMI: %.5lf" % (accuracy, nmi) )

## Visualization of the feature latent space

In [ ]:
# get feature representations
test_features, test_labels = gmvae.latent_features(test_loader, True)

In [ ]:
# import TSNE from scikit-learn library
from sklearn.manifold import TSNE

# reduce dimensionality to 2D, we consider a subset of data because TSNE
# is a slow algorithm
tsne_features = TSNE(n_components=2).fit_transform(test_features[:1000,])

In [ ]:
fig = plt.figure(figsize=(10, 6))

plt.scatter(tsne_features[:, 0], tsne_features[:, 1], c=test_labels[:tsne_features.shape[0]], marker='o',
            edgecolor='none', cmap=plt.cm.get_cmap('jet', 10), s = 10)
plt.grid(False)
plt.axis('off')
plt.colorbar()